# Assistants API 

assistant에게 메세지를 보내면 그 메세지에 따른 출력을 찾아주도록 할 수도 있고 그 도구를 통해서 모은 결과를 답변해줄 수도 있다. 
따라서 우리가 전에 봤던 Agent의 컨셉과 약간 비슷하지만 langchain을 이용해 agent를 구현했습니다. 

이번에는 Langchain을 사용하지 않고 이미 설치되어있는 OpenAI Python 패키지만을 사용할 것입니다. 

Assistant가 가지고 있는 4가지 부분에 주안점을 두자. 


1. Assistant 

2. Thread 

3. Messages 

4. Run 

추가 비용은 발생하지 않음. 

하지만 하위 모델을 사용할 때 GPT3, GPT4, Turbo는 따로 결제해야 합니다. 

(https://platform.openai.com/docs/assistants/overview?context=with-streaming)

Thread는 User와 Assistant가 주고받는 메시지들의 모음입니다. 기본적으로 이건 대화와 같습니다. 
만약 우리가 회사를 위해서 Assistant를 만들었는데 여러 사람들이 그 Assistant를 사용하면, 각각의 사용자가 Thread를 가지게 됩니다. 간단하죠 ? 
Thread에서 수행하는 작업은, message를 추가하는 것입니다. 예를 들어 사용자가 message를 보내면 그걸 thread에 넣어줘야하죠. 
이건 memory를 가질 수 있다는 멋진 점이 있습니다. memory를 저희가 직접 관리할 필요가 없고 자동으로 memory에 메세지를 추가하며 openai가 관리합니다. 
오래된 메세지는 자동으로 지우거나 압축시켜서 용량이 너무 커지지 않게 관리합니다. 그래서 모델의 context window에 알맞도록 하는 것이죠. 
메모리 관리의 모든 것을 처리하면서 history가 model의 context memory에 맞도록 관리합니다. 

즉, Thread는 대화 그 자체입니다. Thread에 메시지를 추가하고 Assistant에게 응답을 받으려면 Thread를 run하는 것입니다. 

사용자의 메세지를 받았다면 Assistant는 function 사용, 파일을 읽고, code interpreter를 사용하던가 하려면 thread를 run 해야합니다.
이게 run의 개념입니다. 단순히 Thread에 메시지를 넣는 것만으로 아무 일도 일어나지 않습니다. 실제로 LLM을 통해서 실행(run)하기 위해서는 
Run을 생성해야 합니다. 

Run이란 OpenAI에게 말하는건데, 기본적으로 Thread안의 메시지를 model을 통해서 실행 해달라는 것입니다. 그러면 model이 그 메세지를 확인하고 그에 대한 reply 하기를 선택하거나, 또는 필요한 툴을 선택할 수 있습니다. 그리고 우리에게 그 툴을 실행할지 물어본 다음에 그 툴을 사용한다면 결과물을 알려줍니다. 

이런 방식으로 질문에 더 잘 대답할 수 있습니다.  그렇게 모든 질문에 대해서 이 과정을 반복합니다. 

정리하자면, Assistant는 파일도 포함할 수 있고 function도 갖춘 AI라고 보시면 됩니다. thread는 사용자와 Assistant간의 대화였습니다. 
Run은 Thread를 execute(실행)하는 방식입니다. 우리가 LLM에게 메세지를 읽고 어떻게 할지 결정해달라고 요청하는 것입니다. 

LLM이 메세지를 읽은 다음에 "이 툴을 사용해야해요"라고 대답할 것입니다. 

RUN -> Thread -> RUN 이 무한 반복되는 것입니다. 

In [17]:
!pip install yfinance openai --upgrage # 모듈의 최신 상태 확인 


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: --upgrage


In [1]:
# tool을 implement 해보자

from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper

def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

In [3]:
import openai as client

# 현재 assistant는 beta 버전으로 사용 가능함. (공식문서 확인)

assistant = client.beta.assistants.create(
    name="Investor Assistant",
    instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
    model="gpt-4-turbo",
    tools = functions,
)

# tools가 어떻게 작동될 지 여러가지 description을 List 형태로 넘겨줄건데, 그 전에 tools를 먼저 만든 다음에 model을 설명해줘야 합니다. 그래야 어떤 tool을 call할 지 알 수 있습니다.

# Investor Assistant 

openai assistant(https://platform.openai.com/assistants) 사이트로 가면 위에서 설정한 "Investor Assistant" 라는 Assistant가 생성된 것을 확인하실 수 있습니다. 자세히 보면 우리가 만든 function인 get_ticker도 있습니다. 